In [1]:
import os
import re
import pandas as pd
import numpy as np
import sys

We have given 2000 document files which belongs to 20 classes.First we load the data set and removing all the stop words from the each file to get the pure word.

In [2]:
stop_words = ["a", "about", "above", "above", "across", "after", "afterwards", "again", "against", "all", "almost", "alone", "along", "already", "also","although","always","am","among", "amongst", "amoungst", "amount",  "an", "and", "another", "any","anyhow","anyone","anything","anyway", "anywhere", "are", "around", "as",  "at", "back","be","became", "because","become","becomes", "becoming", "been", "before", "beforehand", "behind", "being", "below", "beside","the" ,"besides", "between", "beyond", "bill", "both", "bottom","but", "by", "call", "can", "cannot", "cant", "co", "con", "could", "couldnt", "cry", "de", "describe", "detail", "do", "done", "down", "due", "during", "each", "eg", "eight", "either", "eleven","else", "elsewhere", "empty", "enough", "etc", "even", "ever", "every", "everyone", "everything", "everywhere", "except", "few", "fifteen", "fify", "fill", "find", "fire", "first", "five", "for", "former", "formerly", "forty", "found", "four", "from", "front", "full", "further", "get", "give", "go", "had", "has", "hasnt", "have", "he", "hence", "her", "here", "hereafter", "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his", "how", "however", "hundred", "ie", "if", "in", "inc", "indeed", "interest", "into", "is", "it", "its", "itself", "keep", "last", "latter", "latterly", "least", "less", "ltd", "made", "many", "may", "me", "meanwhile", "might", "mill", "mine", "more", "moreover", "most", "mostly", "move", "much", "must", "my", "myself", "name", "namely", "neither", "never", "nevertheless", "next", "nine", "no", "nobody", "none", "noone", "nor", "not", "nothing", "now", "nowhere", "of", "off", "often", "on", "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our", "ours", "ourselves", "out", "over", "own","part", "per", "perhaps", "please", "put", "rather", "re", "same", "see", "seem", "seemed", "seeming", "seems", "serious", "several", "she", "should", "show", "side", "since", "sincere", "six", "sixty", "so", "some", "somehow", "someone", "something", "sometime", "sometimes", "somewhere", "still", "such", "system", "take", "ten", "than", "that", "the", "their", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "therefore", "therein", "thereupon", "these", "they", "thickv", "thin", "third", "this", "those", "though", "three", "through", "throughout", "thru", "thus", "to", "together", "too", "top", "toward", "towards", "twelve", "twenty", "two", "un", "under", "until", "up", "upon", "us", "very", "via", "was", "we", "well", "were", "what", "whatever", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", "wherein", "whereupon", "wherever", "whether", "which", "while", "whither", "who", "whoever", "whole", "whom", "whose", "why", "will", "with", "within", "without", "would", "yet", "you", "your", "yours", "yourself", "yourselves", "the","i"]

In [3]:
word_feature=[] 
dict_features={} 
#Loading the data set
ls=os.listdir('./20_newsgroups1')          
for folders in ls:
    files= os.listdir('./20_newsgroups1/'+folders) 
    word_feature.append(folders)
    for file in files:
        f=open('./20_newsgroups1/'+folders+'/'+file, 'r', encoding="ISO-8859-1")
        text=f.read()
        tokens=re.compile('\w+').findall(text)
        words=[token.lower() for token in tokens if(token.isalpha() and len(token)>1 and token not in stop_words)]   
        for word in words:
            if word in dict_features:
                dict_features[word]+=1
            else:
                    dict_features[word]=1  
dict_features_copy=dict_features                    
dict_features_copy=sorted(dict_features_copy.items(),key=lambda x:x[1],reverse=True) 
#we could have use hole dictionary for the feature but we have  lot of features that don't occurs in our data 
#using top 2000 words which have higher frequency.Lot of words have frequency 1,2,3 so we get rid of these words
col=dict_features_copy[0:2000] 
col=dict(col) 

In [4]:
feature= []
for key, value in col.items():
    feature.append(key) 
#Creating empty dataframe with all features.we would be using words as feature
df=pd.DataFrame(columns=feature)
for folders in ls:
    files= os.listdir('./20_newsgroups1/'+folders) 
#Going through all document built the dictionary     
    for file in files:
        f=open('./20_newsgroups1/'+folders+'/'+file, 'r', encoding="ISO-8859-1")
        text=f.read()
        tokens=re.compile('\w+').findall(text)
        words=[token.lower() for token in tokens if(token.isalpha() and len(token)>1 and token not in stop_words)]
        arr=np.zeros(2000)
#For each document creating emprty numpy zero array with size of 2000
        for word in words:
            if word in feature:
                arr[feature.index(word)]+=1
        df1=pd.DataFrame([arr],columns=feature)        
        df=df.append(df1,ignore_index=True)

In [5]:
df=pd.read_csv('textclassification_data.csv')

In [6]:
#Creating target value Y  
Y=[]
for i in word_feature:
    if i=="soc.religion.christian":
        for j in range(997):
            Y.append(i)
    else:
        for j in range(1000):
            Y.append(i)   

In [7]:
del df['Unnamed: 0']

In [8]:
target=pd.read_csv("textclassification_target.csv",header=None)
Y=target

In [9]:
#Training Naive Bayes (NB) classifier on training data.
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

In [10]:
#spliting the data 
X_train,X_test,Y_train,Y_test=train_test_split(df,Y,test_size=0.20,random_state=1) 

In [11]:
X_train.shape

(15997, 2000)

In [12]:
#performance of multinomial naive bayes
clf=MultinomialNB()
clf.fit(X_train,Y_train)
clf.score(X_train,Y_train)

/home/deepesh/ENTER/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.85922360442582979

In [13]:
clf.score(X_test,Y_test)

0.82825000000000004

In [14]:
Y_pred=clf.predict(X_test)
Y_pred

array(['comp.sys.mac.hardware', 'talk.politics.guns', 'comp.graphics', ...,
       'misc.forsale', 'sci.space', 'misc.forsale'],
      dtype='<U24')

In [15]:
from sklearn.metrics import confusion_matrix,classification_report 
print(classification_report(Y_test, Y_pred))  
print(confusion_matrix(Y_test,Y_pred)) 

                          precision    recall  f1-score   support

             alt.atheism       0.81      0.81      0.81       215
           comp.graphics       0.75      0.74      0.75       170
 comp.os.ms-windows.misc       0.96      0.12      0.22       205
comp.sys.ibm.pc.hardware       0.61      0.84      0.70       195
   comp.sys.mac.hardware       0.73      0.86      0.79       196
          comp.windows.x       0.63      0.84      0.72       201
            misc.forsale       0.80      0.93      0.86       192
               rec.autos       0.85      0.90      0.88       202
         rec.motorcycles       0.87      0.94      0.90       200
      rec.sport.baseball       0.92      0.90      0.91       201
        rec.sport.hockey       0.93      0.90      0.91       196
               sci.crypt       0.97      0.93      0.95       193
         sci.electronics       0.84      0.89      0.87       175
                 sci.med       0.95      0.89      0.92       204
         